<a href="https://colab.research.google.com/github/danielreinaux/DataPrep/blob/main/Titanic_DataPrep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dataset Titanic - Projeto DataPrep:

Neste projeto, nosso foco será exclusivamente a preparação dos dados (DataPrep) para garantir que estejam prontos para análise e modelagem. Não realizaremos análises descritivas nem estatísticas nesta etapa. Além disso, como nosso conjunto de dados já está separado em conjuntos de treino e teste, abordaremos apenas a preparação dos dados no nosso dataset de treino, que já está separado. Todo o processo de DataPrep será realizado diretamente neste conjunto de dados de treino.



Nossa Preparação dos Dados consistirá em:
* Análise dos Metadados
* Tratando nulidade
* Análise de Cardinalidade
* One Hot/Label Encoding
* Normalização/Padronização



In [1]:
import pandas as pd

df = pd.read_csv("/content/train.csv", encoding = 'utf8')

df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### Separando colunas:

1. Alvo Target:
  * O alvo (target) não será tratado durante a preparação dos dados. Ele será separado e reinserido após a conclusão das etapas de preparação.

2. Retirando classes que não fazem sentido para o modelo:
  * Identificadores únicos, como **PassengerID** e **Ticket**, serão removidos, pois não acrescentam valor preditivo ao modelo e podem introduzir ruído.


In [2]:
# Lista de variáveis para retirarmos do tratamento:

lista_ret = ['PassengerId', 'Survived', 'Ticket']

df_00 = df.drop(axis=1, columns=lista_ret)

df_00.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,NaN,S


## Construindo variáveis úteis

No nosso caso, o nome em si não traz informações relevantes, mas podemos extrair o título (Miss, Mrs, Mr, Dr) dos nomes, o que pode ser útil para o modelo

In [3]:
df_00['Titulo'] = df_00['Name'].str.extract(r'(Mr\.|Miss\.|Mrs\.|Dr\.)')
df_00.drop(axis=1, columns='Name', inplace=True)
df_00

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Titulo
0,3,male,22.0,1,0,7.2500,NaN,S,Mr.
1,1,female,38.0,1,0,71.2833,C85,C,Mrs.
2,3,female,26.0,0,0,7.9250,NaN,S,Miss.
3,1,female,35.0,1,0,53.1000,C123,S,Mrs.
4,3,male,35.0,0,0,8.0500,NaN,S,Mr.
...,...,...,...,...,...,...,...,...,...
886,2,male,27.0,0,0,13.0000,NaN,S,NaN
887,1,female,19.0,0,0,30.0000,B42,S,Miss.
888,3,female,NaN,1,2,23.4500,NaN,S,Miss.
889,1,male,26.0,0,0,30.0000,C148,C,Mr.


In [4]:
def generate_metadata(dataframe):
    """
    Gera um dataframe contendo metadados das colunas do dataframe fornecido.

    :param dataframe: DataFrame para o qual os metadados serão gerados.
    :return: DataFrame contendo metadados.
    """

    # Coleta de metadados básicos
    metadata = pd.DataFrame({
        'nome_variavel': dataframe.columns,
        'tipo': dataframe.dtypes,
        'qt_nulos': dataframe.isnull().sum(),
        'percent_nulos': round((dataframe.isnull().sum() / len(dataframe))* 100,2),
        'cardinalidade': dataframe.nunique(),
    })
    metadata=metadata.sort_values(by='tipo')
    metadata = metadata.reset_index(drop=True)

    return metadata

metadata_df = generate_metadata(df_00)
metadata_df

,nome_variavel,tipo,qt_nulos,percent_nulos,cardinalidade
0,Pclass,int64,0,0.00,3
1,SibSp,int64,0,0.00,7
2,Parch,int64,0,0.00,7
3,Age,float64,177,19.87,88
4,Fare,float64,0,0.00,248
5,Sex,object,0,0.00,2
6,Cabin,object,687,77.10,147
7,Embarked,object,2,0.22,3
8,Titulo,object,60,6.73,4


### Tratando Nulidade:

* No nosso dataset, a coluna **Cabin** possui 77% de valores nulos. Devido à alta porcentagem de valores ausentes, decidimos remover essa coluna


* Para as demais variáveis, aplicaremos diferentes métodos de tratamento de valores nulos:
  - Variáveis numéricas: Preencheremos os valores nulos com a média.
  - Variáveis Categóricas: Preencheremos os valores nulos com a moda (o valor que mais aparece).

In [33]:
# Tirando a coluna com muitos nulos
df_00.drop(axis=1, columns='Cabin', inplace=True)

# Preenchendo a idade (categoria numérica) pela média
df_00['Age'].fillna(df['Age'].mean(), inplace=True)

#Preenchendo as colunas categóricas com algum nulo
df_00['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)
df_00['Titulo'].fillna(df_00['Titulo'].mode()[0], inplace=True) # A moda retorna uma lista de valores possivelmente

df_00

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Titulo
0,3,male,22.000000,1,0,7.2500,S,Mr.
1,1,female,38.000000,1,0,71.2833,C,Mrs.
2,3,female,26.000000,0,0,7.9250,S,Miss.
3,1,female,35.000000,1,0,53.1000,S,Mrs.
4,3,male,35.000000,0,0,8.0500,S,Mr.
...,...,...,...,...,...,...,...,...
886,2,male,27.000000,0,0,13.0000,S,Mr.
887,1,female,19.000000,0,0,30.0000,S,Miss.
888,3,female,29.699118,1,2,23.4500,S,Miss.
889,1,male,26.000000,0,0,30.0000,C,Mr.


## Aplicando OneHotEncoder/LabelEncoder

Realizaremos a codificação das variáveis categóricas antes da normalização/padronização, pois os valores transformados pelos encoders também precisam ser normalizados.

#### 1. OneHotEncoder

Aplicaremos o OneHotEncoder nas colunas categóricas com cardinalidade (número de categorias únicas) menor ou igual a um valor de corte específico. Esse método é ideal para variáveis categóricas com poucas categorias, evitando a introdução de ordens arbitrárias nos dados.

In [36]:
from sklearn.preprocessing import OneHotEncoder

cut_off = 20

# Selecionar colunas categóricas
categorical_cols = df_00.select_dtypes(include=['object']).columns

cols_to_onehot = [col for col in categorical_cols if df_00[col].nunique() < cut_off]

encoder = OneHotEncoder(sparse_output=False, drop='first')
encoded_cols = encoder.fit_transform(df_00[cols_to_onehot])

# Obtendo os nomes das colunas resultantes
encoded_cols_names = encoder.get_feature_names_out(cols_to_onehot)

# Criando um DataFrame com as colunas codificadas, preservando os índices originais
encoded_df = pd.DataFrame(encoded_cols,columns = encoded_cols_names, index = df.index)

# Concatenando o DataFrame criando com o nosso
df_01 = pd.concat([df_00.drop(columns=cols_to_onehot), encoded_df], axis=1)

df_01

,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S,Titulo_Miss.,Titulo_Mr.,Titulo_Mrs.
0,3,22.000000,1,0,7.2500,1.0,0.0,1.0,0.0,1.0,0.0
1,1,38.000000,1,0,71.2833,0.0,0.0,0.0,0.0,0.0,1.0
2,3,26.000000,0,0,7.9250,0.0,0.0,1.0,1.0,0.0,0.0
3,1,35.000000,1,0,53.1000,0.0,0.0,1.0,0.0,0.0,1.0
4,3,35.000000,0,0,8.0500,1.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
886,2,27.000000,0,0,13.0000,1.0,0.0,1.0,0.0,1.0,0.0
887,1,19.000000,0,0,30.0000,0.0,0.0,1.0,1.0,0.0,0.0
888,3,29.699118,1,2,23.4500,0.0,0.0,1.0,1.0,0.0,0.0
889,1,26.000000,0,0,30.0000,1.0,0.0,0.0,0.0,1.0,0.0


#### 2.Label Encoder

Utilizaremos o LabelEncoder para colunas categóricas cuja cardinalidade exceda o valor de corte (no nosso caso, 20). Este método é recomendado para situações em que há muitas categorias, convertendo cada categoria em um valor numérico distinto.

In [37]:
from sklearn.preprocessing import LabelEncoder

# Selecionar colunas categóricas
categorical_cols = df_00.select_dtypes(include=['object']).columns

cols_to_onehot = [col for col in categorical_cols if df_00[col].nunique() >= cut_off]

cols_to_onehot


[]

Como não temos nenhuma coluna, continuaremos nosso processo com as próximas etapas

## Normalização / Padronização


Como não estamos focados na resolução do problema com um algoritmo específico e nem pretendemos realizar seleção de características (feature selection), podemos optar entre os processos de normalização e padronização. Para este projeto, escolhemos aplicar a **normalização** aos nossos dados.

In [38]:
# Normalizando nossa tabela
from sklearn.preprocessing import StandardScaler

numeric_cols = df_01.select_dtypes(include=['float64', 'int64', 'int32']).columns

scaler = StandardScaler()
df_01[numeric_cols] = scaler.fit_transform(df_01[numeric_cols])

df_01

,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S,Titulo_Miss.,Titulo_Mr.,Titulo_Mrs.
0,0.827377,-0.592481,0.432793,-0.473674,-0.502445,0.737695,-0.307562,0.615838,-0.506655,0.737695,-0.403962
1,-1.566107,0.638789,0.432793,-0.473674,0.786845,-1.355574,-0.307562,-1.623803,-0.506655,-1.355574,2.475480
2,0.827377,-0.284663,-0.474545,-0.473674,-0.488854,-1.355574,-0.307562,0.615838,1.973729,-1.355574,-0.403962
3,-1.566107,0.407926,0.432793,-0.473674,0.420730,-1.355574,-0.307562,0.615838,-0.506655,-1.355574,2.475480
4,0.827377,0.407926,-0.474545,-0.473674,-0.486337,0.737695,-0.307562,0.615838,-0.506655,0.737695,-0.403962
...,...,...,...,...,...,...,...,...,...,...,...
886,-0.369365,-0.207709,-0.474545,-0.473674,-0.386671,0.737695,-0.307562,0.615838,-0.506655,0.737695,-0.403962
887,-1.566107,-0.823344,-0.474545,-0.473674,-0.044381,-1.355574,-0.307562,0.615838,1.973729,-1.355574,-0.403962
888,0.827377,0.000000,0.432793,2.008933,-0.176263,-1.355574,-0.307562,0.615838,1.973729,-1.355574,-0.403962
889,-1.566107,-0.284663,-0.474545,-0.473674,-0.044381,0.737695,-0.307562,-1.623803,-0.506655,0.737695,-0.403962
